In [75]:
#Update your token
STUDENT_TOKEN = 'TOMAS MESALLES'

Server communication game and game_loop.

In [76]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
    def __init__(self, state, status, player):
        self.state = state
        self.status = status
        self.player = player

    def is_waiting(self):
        return self.status == 'waiting'

    def is_end(self):
        return self.status == 'complete'

    def get_board(self):
        print(self.state)
        return json.loads(self.state)

    def actions(self):
        return []

    def print_game(self):
        print(self.state)

def new_game(game_type, multi_player = False):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
        if r.status_code == 200:
            return r.json()['game-id']
        print(r.content)

def join_game(game_type, game_id):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
        if r.status_code == 200:
          return r.json()['player']
        print(r.content)

def game_state(game_type, game_id, GameClass, debug_mode=False):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
        if r.status_code == 200:
           return GameClass(r.json()['state'], r.json()['status'], r.json()['player'], debug_mode)
        print(r.content)

def update_game(game_type, game_id, player, move):
    for _ in range(10):
        r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
        if r.status_code == 200:
          return r.content
        print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None, debug_mode=False):
    while id == None:
        print('\033[93mCreating new game...\033[0m')
        id = new_game(game_type, multi_player)

    print('\033[93mJoining game with id: \033[0m%s' % id)
    player = join_game(game_type, id)

    print('\033[93mPlaying as %s\033[0m' % player)

    game = game_state(game_type, id, GameClass, debug_mode)
    print('\033[93mWaiting for the other player to join...\033[0m')
    while game.is_waiting():
        time.sleep(10)
        game = game_state(game_type, id, GameClass)

    while True:
        game = game_state(game_type, id, GameClass, debug_mode)
        game.print_game()
        if game.is_end():
            if game.player == '-':
                print('\033[94mDRAW\033[0m')
            else:
                print('\033[92mWINNER\033[0m' if game.player == player else '\033[91mLOSER\033[0m')
            return
        if game.player == player:
            if not debug_mode: print('Making next move...')
            move = solver(game)
            if debug_mode: print(f"LAST MOVE: {move}")
            update_result = update_game(game_type, id, player, json.dumps(move))
            # print(f"UPDATE RESULT: {update_result}")
        else:
            time.sleep(2)

Random solver

In [77]:
def random_solver(game):
    actions = game.actions()
    action = random.choice(actions)
    return action

Game specific code

In [78]:
from functools import reduce
from copy import copy, deepcopy
import json

class NineMensMorris(Game):
    def __init__(self, state, status, player, debug_mode=False):
        super().__init__(state, status, player)
        self.debug_mode = debug_mode
        self.board = state.get('board', ['.'] * 24)
        self.actions_list = state.get('actions', [])
        self.pieces = self.count_pieces()

    def count_pieces(self):
        x_count = self.board.count('X')
        o_count = self.board.count('O')
        return {'X': x_count, 'O': o_count}

    def update_pieces(self):
        self.pieces = self.count_pieces()

    def actions(self):
        return self.state['actions']

    def print_board(self):
        if len(self.board) != 24:
            print("Error: The list must contain exactly 24 elements.")
            return

        # Layout of the board using indices
        board_layout = [
            [0,    None, None, 1,    None, None, 2   ],
            [None, 8,    None, 9,    None, 10,   None],
            [None, None, 16,   17,   18,   None, None],
            [7,    15,   23,   None, 19,   11,   3   ],
            [None, None, 22,   21,   20,   None, None],
            [None, 14,   None, 13,   None, 12,   None],
            [6,    None, None, 5,    None, None, 4   ]
        ]

        # Loop through each row and print the elements
        for row in board_layout:
            line = []
            for index in row:
                if index is None:
                    line.append(" ")
                else:
                    if self.board[index] == self.player:
                        line.append(f"\033[92m{self.board[index]}\033[0m")
                    elif self.board[index] in ['X', 'O']:
                        line.append(f"\033[91m{self.board[index]}\033[0m")
                    else:
                        line.append(".")
            print(" ".join(line))

    def print_game(self):
        print(f"Board:")
        self.print_board()
        if self.debug_mode: print(f"ACTIONS: {self.actions_list}")
        #print(self.board)
        if not self.debug_mode: print(f"Player: {self.player}")
        if self.debug_mode: print(f"PIECES: {self.pieces}")
        if not self.debug_mode: print(f"Phase: {self.phase}")
        print()

    def other_player(self):
        if self.player == 'X': return 'O'
        if self.player == 'O': return 'X'

In [79]:
game_loop(random_solver, NineMensMorris, 'ninemensmorris', multi_player=False, id=None, debug_mode=True)

Creating new game...
Joining game with id: 21491
Playing as O
Waiting for the other player to join...
Board:
.     .     .
  .   .   .  
    . . .    
. . .   X . .
    . . .    
  .   .   .  
.     .     .
ACTIONS: [[-1, 0, -1], [-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 13, -1], [-1, 14, -1], [-1, 15, -1], [-1, 16, -1], [-1, 17, -1], [-1, 18, -1], [-1, 20, -1], [-1, 21, -1], [-1, 22, -1], [-1, 23, -1]]
PIECES: {'X': 1, 'O': 0}

LAST MOVE: [-1, 15, -1]
Board:
.     .     .
  .   .   .  
    X . .    
. O .   X . .
    . . .    
  .   .   .  
.     .     .
ACTIONS: [[-1, 0, -1], [-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 13, -1], [-1, 14, -1], [-1, 17, -1], [-1, 18, -1], [-1, 20, -1], [-1, 21, -1], [-1, 22, -1], [-1, 23, -1]]
PIECES: {'X': 2, 'O': 

# Alpha-Beta Solver

In [80]:
class NineMensMorris(Game):
    def __init__(self, state, status, player, debug_mode=False):
        super().__init__(state, status, player)
        self.debug_mode = debug_mode
        self.board = state.get('board', ['.'] * 24)
        self.actions_list = state.get('actions', [])
        self.pieces = self.count_pieces()
        self.phase = state.get('phase', 'placing')

    def count_pieces(self):
        x_count = self.board.count('X')
        o_count = self.board.count('O')
        return {'X': x_count, 'O': o_count}

    def update_pieces(self):
        self.pieces = self.count_pieces()

    def actions(self):
        return self.state['actions']

    def print_board(self):
        if len(self.board) != 24:
            print("Error: The list must contain exactly 24 elements.")
            return

        # Layout of the board using indices
        board_layout = [
            [0,    None, None, 1,    None, None, 2   ],
            [None, 8,    None, 9,    None, 10,   None],
            [None, None, 16,   17,   18,   None, None],
            [7,    15,   23,   None, 19,   11,   3   ],
            [None, None, 22,   21,   20,   None, None],
            [None, 14,   None, 13,   None, 12,   None],
            [6,    None, None, 5,    None, None, 4   ]
        ]

        # Loop through each row and print the elements
        for row in board_layout:
            line = []
            for index in row:
                if index is None:
                    line.append(" ")
                else:
                    if self.board[index] == self.player:
                        line.append(f"\033[92m{self.board[index]}\033[0m")
                    elif self.board[index] in ['X', 'O']:
                        line.append(f"\033[91m{self.board[index]}\033[0m")
                    else:
                        line.append(".")
            print(" ".join(line))

    def print_game(self):
        print(f"Board:")
        self.print_board()
        if self.debug_mode: print(f"ACTIONS: {self.actions_list}")
        #print(self.board)
        if not self.debug_mode: print(f"Player: {self.player}")
        if self.debug_mode: print(f"PIECES: {self.pieces}")
        if not self.debug_mode: print(f"Phase: {self.phase}")
        print()

    def other_player(self):
        if self.player == 'X': return 'O'
        if self.player == 'O': return 'X'

    def is_terminal(self):
        return self.phase == 'game_over' or not self.actions_list


In [81]:

MAX_DEPTH = 5

def alpha_beta_solver(game):
    """
    Alpha-beta pruning solver for Nine Men's Morris with optimized depth and heuristics.
    """
    def max_value(state, alpha, beta, depth):
        if depth >= MAX_DEPTH or state.is_terminal():
            return evaluate_state(state), None
        value, best_action = float('-inf'), None
        for action in sorted(state.actions(), key=lambda x: evaluate_action(state, x), reverse=True):
            new_state = simulate_action(state, action)
            min_val, _ = min_value(new_state, alpha, beta, depth + 1)
            if min_val > value:
                value, best_action = min_val, action
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return value, best_action

    def min_value(state, alpha, beta, depth):
        if depth >= MAX_DEPTH or state.is_terminal():
            return evaluate_state(state), None
        value, best_action = float('inf'), None
        for action in sorted(state.actions(), key=lambda x: evaluate_action(state, x)):
            new_state = simulate_action(state, action)
            max_val, _ = max_value(new_state, alpha, beta, depth + 1)
            if max_val < value:
                value, best_action = max_val, action
            beta = min(beta, value)
            if beta <= alpha:
                break
        return value, best_action

    _, best_action = max_value(game, float('-inf'), float('inf'), 0)
    return best_action

def evaluate_state(state):
    """
    Heuristic evaluation function.
    - Prioritize pieces and mills.
    """
    player = state.player
    opponent = 'O' if player == 'X' else 'X'
    player_pieces = state.pieces[player]
    opponent_pieces = state.pieces[opponent]
    return 10 * player_pieces - 8 * opponent_pieces

def evaluate_action(state, action):
    """
    Heuristic to prioritize actions based on potential gain.
    """
    # Actions forming mills are prioritized
    new_state = simulate_action(state, action)
    return evaluate_state(new_state)

def simulate_action(state, action):
    """
    Simulates an action and returns the updated state.
    """
    new_state = deepcopy(state)
    if isinstance(action, dict):
        pos_from = action.get('from', -1)
        pos_to = action.get('to', -1)
        if pos_from >= 0:
            new_state.board[pos_from] = '.'
        new_state.board[pos_to] = state.player
    return new_state

game_loop(alpha_beta_solver, NineMensMorris, 'ninemensmorris', multi_player=False, id=None, debug_mode=True)

Creating new game...
Joining game with id: 21492
Playing as X
Waiting for the other player to join...
Board:
.     .     .
  .   .   .  
    . . .    
. . .   . . .
    . . .    
  .   .   .  
.     .     .
ACTIONS: [[-1, 0, -1], [-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 13, -1], [-1, 14, -1], [-1, 15, -1], [-1, 16, -1], [-1, 17, -1], [-1, 18, -1], [-1, 19, -1], [-1, 20, -1], [-1, 21, -1], [-1, 22, -1], [-1, 23, -1]]
PIECES: {'X': 0, 'O': 0}

LAST MOVE: [-1, 0, -1]
Board:
X     .     .
  .   .   .  
    . . .    
. O .   . . .
    . . .    
  .   .   .  
.     .     .
ACTIONS: [[-1, 1, -1], [-1, 2, -1], [-1, 3, -1], [-1, 4, -1], [-1, 5, -1], [-1, 6, -1], [-1, 7, -1], [-1, 8, -1], [-1, 9, -1], [-1, 10, -1], [-1, 11, -1], [-1, 12, -1], [-1, 13, -1], [-1, 14, -1], [-1, 16, -1], [-1, 17, -1], [-1, 18, -1], [-1, 19, -1], [-1, 20, -1], [-1, 21, -1], [-1, 22, -1], [-1, 23,